# with MDAnalysis 2.6.1

In [6]:
import pandas as pd
import numpy as np
import MDAnalysis as mda
import psutil

In [7]:
pd.__version__

'2.1.0'

In [8]:
np.__version__

'1.22.4'

In [9]:
mda.__version__

'2.6.1'

In [10]:
psutil.__version__

'5.9.0'

# Load into MDAnalysis Universe

In [11]:
tprfile= "data/1AKI_prod_60.tpr"
trrfile= "data/1AKI_prod_60.trr"
u=mda.Universe(tprfile,trrfile)

In [12]:
u

<Universe with 33876 atoms>

In [13]:
#struc=u.select_atoms("all")
#struc.write("lysozyme.pdb") # .tpr to .pdb file

# Trim the data to reduce analysis time

In [14]:
from CodeEntropy.IO import MDAUniverseHelper as MDAHelper
selection_string_pre_process= 'protein'
start = 3
end = 40
step = 1

reduced_frame= MDAHelper.new_U_select_frame(u,start,end, step)

reduced_atom=MDAHelper.new_U_select_atom(reduced_frame, selection_string_pre_process)

In [15]:
reduced_atom

<Universe with 1960 atoms>

# Parse the data into CodeEntropy Object

In [16]:
from CodeEntropy.ClassCollection import DataContainer as DC
dataContainer = DC.DataContainer (reduced_atom)

Number of atoms      : 1960
Number of frames     : 37


# Calculations

In [17]:
from CodeEntropy.FunctionCollection import EntropyFunctions as EF
selection_string="all"
axis_list=["C5'","C4'","C3'"]
outfile = None
moutFile = None
nmdFile = None
csv_out = None
tScale= 1.0
fScale= 1.0
temper =300.0 #temperature = 300K
verbose =2
thread=4

## Whole molecule/Polymer Level

In [18]:
wm_entropyFF, wm_entropyTT = EF.compute_entropy_whole_molecule_level(
    arg_hostDataContainer = dataContainer,
    arg_outFile = outfile,
    arg_selector = selection_string,
    arg_moutFile = moutFile,
    arg_nmdFile = nmdFile,
    arg_fScale = fScale,
    arg_tScale = tScale,
    arg_temper = temper,
    arg_verbose = verbose
)
print(f"wm_entropyFF={wm_entropyFF}")
print(f"wm_entropyTT={wm_entropyTT}")

------------------------------------------------------------
          Hierarchy level. --> Whole molecule <--           
------------------------------------------------------------
Total number of beads at the whole molecule level = 1
Assigning Translation and Rotation Axes @ whole molecule level-> Done
Updating Local coordinates-> Done
Updating Local forces-> Done
Updating Local torques-> Done
Weighting forces and torques-> Done
Updating the submatrices ... 
Done
Generating Quadrants-> Done
Diagonalizing-> Done
Changing the units of eigen values to SI units-> Done
Sorting spectrum in ascending order of frequencies-> Done
Entropy values:
FF Entropy (Whole mol level)             : 52.7444 J/mol/K
TT Entropy (Whole mol level)             : 49.3962 J/mol/K
wm_entropyFF=52.74442658801501
wm_entropyTT=49.39618814278469


## Residue Level

In [19]:
res_entropyFF, res_entropyTT = EF.compute_entropy_residue_level(
    arg_hostDataContainer = dataContainer,
    arg_outFile = outfile,
    arg_selector = selection_string,
    arg_moutFile = moutFile,
    arg_nmdFile = nmdFile,
    arg_fScale = fScale,
    arg_tScale = tScale,
    arg_temper = temper,
    arg_verbose = verbose,
)
print(f"res_entropyFF={res_entropyFF}")
print(f"res_entropyTT={res_entropyTT}")

------------------------------------------------------------
             Hierarchy level. --> Residues <--              
------------------------------------------------------------
LYS1
VAL2
PHE3
GLY4
ARG5
CYS6
GLU7
LEU8
ALA9
ALA10
ALA11
MET12
LYS13
ARG14
HIS15
GLY16
LEU17
ASP18
ASN19
TYR20
ARG21
GLY22
TYR23
SER24
LEU25
GLY26
ASN27
TRP28
VAL29
CYS30
ALA31
ALA32
LYS33
PHE34
GLU35
SER36
ASN37
PHE38
ASN39
THR40
GLN41
ALA42
THR43
ASN44
ARG45
ASN46
THR47
ASP48
GLY49
SER50
THR51
ASP52
TYR53
GLY54
ILE55
LEU56
GLN57
ILE58
ASN59
SER60
ARG61
TRP62
TRP63
CYS64
ASN65
ASP66
GLY67
ARG68
THR69
PRO70
GLY71
SER72
ARG73
ASN74
LEU75
CYS76
ASN77
ILE78
PRO79
CYS80
SER81
ALA82
LEU83
LEU84
SER85
SER86
ASP87
ILE88
THR89
ALA90
SER91
VAL92
ASN93
CYS94
ALA95
LYS96
LYS97
ILE98
VAL99
SER100
ASP101
GLY102
ASN103
GLY104
MET105
ASN106
ALA107
TRP108
VAL109
ALA110
TRP111
ARG112
ASN113
ARG114
CYS115
LYS116
GLY117
THR118
ASP119
VAL120
GLN121
ALA122
TRP123
ILE124
ARG125
GLY126
CYS127
ARG128
LEU129
Total number of beads 

/home/ioana/Desktop/STFC/CodeEntropy/code/CodeEntropy/CodeEntropy/FunctionCollection/EntropyFunctions.py:47: RuntimeWarning: invalid value encountered in sqrt
  return nmp.sqrt((arg_lambdas)/UAC.get_KT2J(arg_temper))/(2*nmp.pi)
/home/ioana/Desktop/STFC/CodeEntropy/code/CodeEntropy/CodeEntropy/FunctionCollection/EntropyFunctions.py:59: RuntimeWarning: invalid value encountered in sqrt
  afac = UAC.M2ANG * UAC.sqrtKG2AMU * nmp.divide(UAC.get_KT2J(arg_temper), nmp.sqrt(arg_lambdas))


## United Atom Level 

In [20]:
UA_entropyFF, UA_entropyTT, res_df = EF.compute_entropy_UA_level_multiprocess(
    arg_hostDataContainer = dataContainer,
    arg_outFile = outfile,
    arg_selector = selection_string,
    arg_moutFile = moutFile,
    arg_nmdFile = nmdFile,
    arg_fScale = fScale,
    arg_tScale = tScale,
    arg_temper = temper,
    arg_verbose = verbose,
    arg_csv_out=csv_out,
    arg_thread=thread,
)
print(f"UA_entropyFF = {UA_entropyFF}")
print(f"UA_entropyTT = {UA_entropyTT}")
print("Per residue entropy:")
with pd.option_context('display.max_rows', None,
                      'display.max_columns', None,
                      'display.precision',3,
                      ):
    print(res_df)

A shape change has occured (15,15) -> (7, 7)



A shape change has occured (27,27) -> (20, 20)



A shape change has occured (24,24) -> (10, 10)



A shape change has occured (15,15) -> (7, 7)



A shape change has occured (42,42) -> (19, 19)



A shape change has occured (21,21) -> (12, 12)



A shape change has occured (24,24) -> (13, 13)



A shape change has occured (36,36) -> (17, 17)



A shape change has occured (21,21) -> (12, 12)



A shape change has occured (33,33) -> (17, 17)



A shape change has occured (27,27) -> (19, 19)



A shape change has occured (18,18) -> (7, 7)



A shape change has occured (12,12) -> (5, 5)



A shape change has occured (33,33) -> (21, 21)



A shape change has occured (15,15) -> (7, 7)



A shape change has occured (12,12) -> (5, 5)



A shape change has occured (15,15) -> (7, 7)



A shape change has occured (33,33) -> (21, 21)



A shape change has occured (33,33) -> (21, 21)



A shape change has occured (27,27) -> (19, 19)



A shape change

In [21]:
print(UA_entropyFF)
print(UA_entropyTT)

5260.546411602676
2189.1392033863235


In [22]:
result_topo_BB = EF.compute_topographical_entropy0_BB(
    arg_hostDataContainer = dataContainer,
    arg_selector = selection_string,
    arg_outFile = outfile,
    arg_verbose = verbose
)

print(f"result_topo_BB = {result_topo_BB}")

------------------------------------------------------------
Topographical entropy of BB dihedrals 
 computed using the pLogp formalism
------------------------------------------------------------
Found a total of 886 BB dihedrals.
Dihedral 1262 1277 1279 1281  : -0.0000 (-10000)
Dihedral 316  318  320  340   : -0.0000 (   21)
Dihedral 4    24   22   23    : -0.0000 (-10000)
Dihedral 1495 1494 1496 1498  : -0.0000 (-10000)
Dihedral 636  638  640  646   : 1.7484 (   42)
Dihedral 726  728  730  732   : 1.0331 (   48)
Dihedral 1690 1688 1698 1700  : 4.6128 (  113)
Dihedral 995  1005 1007 1009  : -0.0000 (-10000)
Dihedral 1874 1873 1875 1877  : -0.0000 (-10000)
Dihedral 473  481  479  480   : -0.0000 (-10000)
Dihedral 1722 1724 1726 1732  : -0.0000 (  115)
Dihedral 900  902  909  911   : -0.0000 (   60)
Dihedral 1519 1521 1524 1526  : 4.0329 (  102)
Dihedral 702  712  714  716   : -0.0000 (-10000)
Dihedral 675  674  676  678   : -0.0000 (-10000)
Dihedral 1441 1461 1459 1460  : -0.0000 (-10

Dihedral 1281 1288 1290 1292  : -0.0000 (-10000)
Dihedral 1830 1832 1834 1854  : -0.0000 (  123)
Dihedral 1230 1229 1231 1233  : -0.0000 (-10000)
Dihedral 475  473  479  481   : 2.8480 (   31)
Dihedral 662  664  674  676   : -0.0000 (   44)
Dihedral 1688 1700 1698 1699  : -0.0000 (-10000)
Dihedral 1294 1292 1299 1301  : 1.7484 (   86)
Dihedral 1098 1118 1120 1122  : -0.0000 (-10000)
Dihedral 1580 1586 1588 1590  : -0.0000 (-10000)
Dihedral 256  254  269  271   : -0.0000 (   17)
Dihedral 1698 1700 1702 1722  : 1.0331 (  114)
Dihedral 933  935  937  957   : 3.2928 (   62)
Dihedral 1498 1507 1505 1506  : -0.0000 (-10000)
Dihedral 1580 1588 1586 1587  : -0.0000 (-10000)
Dihedral 991  993  995  1005  : -0.0000 (   65)
Dihedral 1698 1700 1702 1704  : 1.0331 (  114)
Dihedral 1726 1734 1732 1733  : -0.0000 (-10000)
Dihedral 884  886  888  898   : -0.0000 (   59)
Dihedral 867  869  884  886   : -0.0000 (   58)
Dihedral 810  812  814  816   : -0.0000 (   55)
Dihedral 1626 1628 1630 1632  : -0.00

Dihedral 230  245  243  244   : -0.0000 (-10000)
Dihedral 295  297  299  301   : -0.0000 (   20)
Dihedral 1660 1662 1664 1666  : -0.0000 (  112)
Dihedral 1901 1906 1904 1905  : -0.0000 (-10000)
Dihedral 1062 1064 1065 1076  : -0.0000 (   70)
Dihedral 147  155  153  154   : -0.0000 (-10000)
Dihedral 1821 1820 1822 1824  : -0.0000 (-10000)
Dihedral 1344 1346 1348 1354  : -0.0000 (   90)
Dihedral 1506 1505 1507 1509  : -0.0000 (-10000)
Dihedral 902  909  911  913   : -0.0000 (-10000)
Dihedral 296  295  297  299   : -0.0000 (-10000)
Dihedral 1419 1437 1439 1441  : -0.0000 (-10000)
Dihedral 42   60   58   59    : -0.0000 (-10000)
Dihedral 105  103  114  116   : 3.2928 (    7)
Dihedral 495  493  511  513   : 2.3397 (   33)
Dihedral 368  370  372  379   : -0.0000 (   24)
Dihedral 1366 1365 1367 1369  : -0.0000 (-10000)
Dihedral 1761 1763 1765 1775  : -0.0000 (  118)
Dihedral 22   24   26   38    : -0.0000 (    2)
Dihedral 153  155  157  163   : -0.0000 (   11)
Dihedral 383  398  400  402   : 

# Solvent entropy

In [23]:
from CodeEntropy.ClassCollection.PoseidonClass import Poseidon
poseidon_object=Poseidon(container=u,start=0,end=20)

2023-09-13 10:08:21.228004
frame = 0
< Timestep 0 with unit cell dimensions [69.57307 69.57307 69.57307 90.      90.      90.     ] >
0:00:25.457068
frame = 1
< Timestep 1 with unit cell dimensions [69.476456 69.476456 69.476456 90.       90.       90.      ] >


/home/ioana/Desktop/STFC/CodeEntropy/code/CodeEntropy/CodeEntropy/poseidon/extractData/HBRAD.py:192: RuntimeWarning: invalid value encountered in arccos
  angle1 = np.arccos(cosine_angle)


0:00:47.112315
frame = 2
< Timestep 2 with unit cell dimensions [69.61369 69.61369 69.61369 90.      90.      90.     ] >
0:01:09.316090
frame = 3
< Timestep 3 with unit cell dimensions [69.52697 69.52697 69.52697 90.      90.      90.     ] >
0:01:31.285315
frame = 4
< Timestep 4 with unit cell dimensions [69.57063 69.57063 69.57063 90.      90.      90.     ] >
0:01:52.943425
frame = 5
< Timestep 5 with unit cell dimensions [69.51649 69.51649 69.51649 90.      90.      90.     ] >
0:02:15.128945
frame = 6
< Timestep 6 with unit cell dimensions [69.44309 69.44309 69.44309 90.      90.      90.     ] >
0:02:38.643245
frame = 7
< Timestep 7 with unit cell dimensions [69.6301 69.6301 69.6301 90.     90.     90.    ] >
0:03:02.103905
frame = 8
< Timestep 8 with unit cell dimensions [69.58307 69.58307 69.58307 90.      90.      90.     ] >
0:03:25.226952
frame = 9
< Timestep 9 with unit cell dimensions [69.59987 69.59987 69.59987 90.      90.      90.     ] >
0:03:47.501312
frame = 10
< Ti

## Whole Molecule/Polymer Level

In [24]:
result_wm = poseidon_object.run_analysis(level_list = ['moleculeLevel'], verbose = False, forceUnits="Kcal") # forces value supplied in the trajectory are in kcal
print(result_wm)

2023-09-13 10:26:30.242416

solvent: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

water: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

1. Populate Dictionaries

memory use: 1.391 GB
0:00:00.001976
memory use: 1.396 GB
0:00:21.137774
0:00:21.138198
memory use: 1.396 GB

Total number of frames: 20.0
Number of atoms in each frame: 11645
Number of variables in each list: 19

2. Process Dictionaries
['moleculeLevel']
---level: moleculeLevel


0:00:23.981752
{'moleculeLevel': {'solventData':    nearest assigned  shell_num   variable      value count
0      ALA      SOL        1.0  Sor_test2   2.045613   270
1      ALA      SOL        1.0     Strans  10.853957   270
2      ALA      SOL        1.0       Srot   0.541862   270
3      ALA      SOL        1.0      count  13.500000   270
4      ARG      SOL        1.0  Sor_test2   4.307759  2145
..     ...      ...        ...        ...        ...   ...
79     TYR      SOL        1.0      count   8.450000   169
80     VAL      SOL 

/home/ioana/Desktop/STFC/CodeEntropy/code/CodeEntropy/CodeEntropy/poseidon/analysis/EECalculation.py:859: RuntimeWarning: invalid value encountered in double_scalars
  w = w ** 0.5
/home/ioana/Desktop/STFC/CodeEntropy/code/CodeEntropy/CodeEntropy/poseidon/analysis/EECalculation.py:1515: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  solventData = pd.concat([solventData, newRowSolvent], ignore_index=True)
/home/ioana/Desktop/STFC/CodeEntropy/code/CodeEntropy/CodeEntropy/poseidon/analysis/EECalculation.py:1552: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, 

In [25]:
wm_solvent_df=result_wm['moleculeLevel']['solventData']

In [26]:
print(wm_solvent_df.to_string())

   nearest assigned  shell_num   variable       value count
0      ALA      SOL        1.0  Sor_test2    2.045613   270
1      ALA      SOL        1.0     Strans   10.853957   270
2      ALA      SOL        1.0       Srot    0.541862   270
3      ALA      SOL        1.0      count   13.500000   270
4      ARG      SOL        1.0  Sor_test2    4.307759  2145
5      ARG      SOL        1.0     Strans   12.213378  2145
6      ARG      SOL        1.0       Srot    0.538416  2145
7      ARG      SOL        1.0      count  107.250000  2145
8      ASN      SOL        1.0  Sor_test2    4.417797  1333
9      ASN      SOL        1.0     Strans   12.261870  1333
10     ASN      SOL        1.0       Srot    0.557429  1333
11     ASN      SOL        1.0      count   66.650000  1333
12     ASP      SOL        1.0  Sor_test2    1.406791   809
13     ASP      SOL        1.0     Strans   11.228171   809
14     ASP      SOL        1.0       Srot    0.436731   809
15     ASP      SOL        1.0      coun

In [69]:
wm_solvent_df.to_csv('wm_solvent.csv')

In [70]:
S_or_wm=0
S_trans_wm=0
S_rot_wm=0
for i in range (0,83,4):
    S_or_wm+= wm_solvent_df['value'][i] #4k -> orientational entropy
    S_trans_wm+= wm_solvent_df['value'][i+1] #4k+1 -> translational entropy
    S_rot_wm+= wm_solvent_df['value'][i+2] #4k+2 -> rotational entropy

In [71]:
print(S_or_wm)
print(S_trans_wm)
print(S_rot_wm)
S_total_wm=S_or_wm+S_trans_wm+S_rot_wm
print(S_total_wm)

43.459580791136545
253.77606015082102
12.571777867753129
309.8074188097107


In [29]:
wm_solute_df=result_wm['moleculeLevel']['soluteData']

In [35]:
print(wm_solute_df.to_string())

    resName      variable       value count
0       ALA     WM_Strans    2.237692   240
1       ALA       WM_Srot    1.842312   240
2       ALA  WM_UA_Strans    0.003090   240
3       ALA    WM_UA_Srot    2.528334   240
4       ARG     WM_Strans    6.307841   220
5       ARG       WM_Srot    4.186435   220
6       ARG  WM_UA_Strans    4.421717   220
7       ARG    WM_UA_Srot    1.332549   220
8       ARG       conf_AE   42.385285   220
9       ASN     WM_Strans    4.764232   280
10      ASN       WM_Srot    2.999115   280
11      ASN  WM_UA_Strans    2.109526   280
12      ASN    WM_UA_Srot    0.467818   280
13      ASN       conf_AE   24.279920   280
14      ASP     WM_Strans    3.990985   140
15      ASP       WM_Srot    2.564136   140
16      ASP  WM_UA_Strans    1.725408   140
17      ASP    WM_UA_Srot    0.418705   140
18      ASP       conf_AE   25.935006   140
19       CL     WM_Strans   21.495253   160
20       CL       WM_Srot    0.000000   160
21       CL  WM_UA_Strans    0.0

In [72]:
wm_solute_df.to_csv('wm_solute.csv')

## Residue/Monomer Level

In [36]:
result_res = poseidon_object.run_analysis(level_list = ['residLevel_resname'], verbose = False, forceUnits="Kcal") # forces value supplied in the trajectory are in kcal
print(result_res)

2023-09-13 10:45:22.961968

solvent: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

water: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

1. Populate Dictionaries

memory use: 1.398 GB
0:00:00.001236
memory use: 1.398 GB
0:00:21.387317
0:00:21.387788
memory use: 1.398 GB

Total number of frames: 20.0
Number of atoms in each frame: 11645
Number of variables in each list: 19

2. Process Dictionaries
['residLevel_resname']
---level: residLevel_resname


/home/ioana/Desktop/STFC/CodeEntropy/code/CodeEntropy/CodeEntropy/poseidon/analysis/EECalculation.py:859: RuntimeWarning: invalid value encountered in double_scalars
  w = w ** 0.5
/home/ioana/Desktop/STFC/CodeEntropy/code/CodeEntropy/CodeEntropy/poseidon/analysis/EECalculation.py:1515: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  solventData = pd.concat([solventData, newRowSolvent], ignore_index=True)
/home/ioana/Desktop/STFC/CodeEntropy/code/CodeEntropy/CodeEntropy/poseidon/analysis/EECalculation.py:1552: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, 



0:00:26.441461
{'residLevel_resname': {'solventData':      nearest assigned  shell_num   variable      value count
0     ALA_10      SOL        1.0  Sor_test2   2.050395    19
1     ALA_10      SOL        1.0     Strans   8.909477    19
2     ALA_10      SOL        1.0       Srot   0.379505    19
3     ALA_10      SOL        1.0      count   0.950000    19
4    ALA_107      SOL        1.0  Sor_test2   1.684203    60
..       ...      ...        ...        ...        ...   ...
515    VAL_2      SOL        1.0      count   6.750000   135
516   VAL_99      SOL        1.0  Sor_test2   0.000000    17
517   VAL_99      SOL        1.0     Strans  14.724812    17
518   VAL_99      SOL        1.0       Srot   1.644911    17
519   VAL_99      SOL        1.0      count   0.850000    17

[520 rows x 6 columns], 'soluteData':      resName      variable      value count
0     ALA_10     WM_Strans   1.844334    20
1     ALA_10       WM_Srot   2.242724    20
2     ALA_10  WM_UA_Strans   0.018059    

In [79]:
res_solvent_df=result_res['residLevel_resname']['solventData']

In [39]:
print(res_solvent_df.to_string())

      nearest assigned  shell_num   variable      value count
0      ALA_10      SOL        1.0  Sor_test2   2.050395    19
1      ALA_10      SOL        1.0     Strans   8.909477    19
2      ALA_10      SOL        1.0       Srot   0.379505    19
3      ALA_10      SOL        1.0      count   0.950000    19
4     ALA_107      SOL        1.0  Sor_test2   1.684203    60
5     ALA_107      SOL        1.0     Strans  11.002773    60
6     ALA_107      SOL        1.0       Srot   0.788850    60
7     ALA_107      SOL        1.0      count   3.000000    60
8      ALA_11      SOL        1.0  Sor_test2   2.899096     4
9      ALA_11      SOL        1.0     Strans  16.340877     4
10     ALA_11      SOL        1.0       Srot   0.893548     4
11     ALA_11      SOL        1.0      count   0.200000     4
12    ALA_110      SOL        1.0  Sor_test2   2.307671    18
13    ALA_110      SOL        1.0     Strans  10.742708    18
14    ALA_110      SOL        1.0       Srot   1.039573    18
15    AL

In [80]:
res_solvent_df.to_csv('res_solvent.csv')

In [74]:
S_or_res=0
S_trans_res=0
S_rot_res=0
for i in range (0,519,4):
    S_or_res+= res_solvent_df['value'][i] #4k -> orientational entropy
    S_trans_res+= res_solvent_df['value'][i+1] #4k+1 -> translational entropy
    S_rot_res+= res_solvent_df['value'][i+2] #4k+2 -> rotational entropy

In [41]:
print(S_or_res)
print(S_trans_res)
print(S_rot_res)

146.8826000018816
1799.3510977924984
198.6618170235331


In [42]:
S_total_res=S_or_res+S_trans_res+S_rot_res
print(S_total_res)

2144.895514817913


In [45]:
res_solute_df=result_res['residLevel_resname']['soluteData']

In [46]:
print(res_solute_df.to_string())

      resName      variable       value count
0      ALA_10     WM_Strans    1.844334    20
1      ALA_10       WM_Srot    2.242724    20
2      ALA_10  WM_UA_Strans    0.018059    20
3      ALA_10    WM_UA_Srot    4.074224    20
4     ALA_107     WM_Strans    2.703594    20
5     ALA_107       WM_Srot    2.511976    20
6     ALA_107  WM_UA_Strans    0.008305    20
7     ALA_107    WM_UA_Srot    5.517062    20
8      ALA_11     WM_Strans    2.267578    20
9      ALA_11       WM_Srot    3.679596    20
10     ALA_11  WM_UA_Strans    0.045276    20
11     ALA_11    WM_UA_Srot    6.124936    20
12    ALA_110     WM_Strans    3.387778    20
13    ALA_110       WM_Srot    2.270880    20
14    ALA_110  WM_UA_Strans    0.036217    20
15    ALA_110    WM_UA_Srot    5.680146    20
16    ALA_122     WM_Strans    3.323822    20
17    ALA_122       WM_Srot    2.605348    20
18    ALA_122  WM_UA_Strans    0.013548    20
19    ALA_122    WM_UA_Srot    5.493574    20
20     ALA_31     WM_Strans    4.2

In [81]:
res_solute_df.to_csv('res_solute.csv')

## United Atom Level

In [47]:
result_ua = poseidon_object.run_analysis(level_list = ['atomLevel'], verbose = False, forceUnits="Kcal") # forces value supplied in the trajectory are in kcal
print(result_ua)

2023-09-13 10:50:00.506169

solvent: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

water: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

1. Populate Dictionaries

memory use: 1.398 GB
0:00:00.001436
memory use: 1.398 GB
0:00:18.828199
0:00:18.828671
memory use: 1.398 GB

Total number of frames: 20.0
Number of atoms in each frame: 11645
Number of variables in each list: 19

2. Process Dictionaries
['atomLevel']
---level: atomLevel


/home/ioana/Desktop/STFC/CodeEntropy/code/CodeEntropy/CodeEntropy/poseidon/analysis/EECalculation.py:859: RuntimeWarning: invalid value encountered in double_scalars
  w = w ** 0.5
/home/ioana/Desktop/STFC/CodeEntropy/code/CodeEntropy/CodeEntropy/poseidon/analysis/EECalculation.py:1515: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  solventData = pd.concat([solventData, newRowSolvent], ignore_index=True)
/home/ioana/Desktop/STFC/CodeEntropy/code/CodeEntropy/CodeEntropy/poseidon/analysis/EECalculation.py:1552: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, 



0:00:23.774986
{'atomLevel': {'solventData':     nearest assigned  shell_num   variable      value count
0     ALA_C    SOL_O        1.0  Sor_test2   2.550446   150
1     ALA_C    SOL_O        1.0     Strans  10.376692   150
2     ALA_C    SOL_O        1.0       Srot   0.520235   150
3     ALA_C    SOL_O        1.0      count   7.500000   150
4     ALA_N    SOL_O        1.0  Sor_test2   0.115634    12
..      ...      ...        ...        ...        ...   ...
231   VAL_N    SOL_O        1.0      count   1.450000    29
232   VAL_O    SOL_O        1.0  Sor_test2   0.204168    41
233   VAL_O    SOL_O        1.0     Strans  12.410060    41
234   VAL_O    SOL_O        1.0       Srot   0.851809    41
235   VAL_O    SOL_O        1.0      count   2.050000    41

[236 rows x 6 columns], 'soluteData':     resName      variable      value count
0     ALA_N     WM_Strans   2.237692   240
1     ALA_N       WM_Srot   1.842312   240
2     ALA_N  WM_UA_Strans   0.003090   240
3     ALA_N    WM_UA_S

In [48]:
ua_solvent_df=result_ua['atomLevel']['solventData']

In [49]:
print(ua_solvent_df.to_string())

    nearest assigned  shell_num   variable      value count
0     ALA_C    SOL_O        1.0  Sor_test2   2.550446   150
1     ALA_C    SOL_O        1.0     Strans  10.376692   150
2     ALA_C    SOL_O        1.0       Srot   0.520235   150
3     ALA_C    SOL_O        1.0      count   7.500000   150
4     ALA_N    SOL_O        1.0  Sor_test2   0.115634    12
5     ALA_N    SOL_O        1.0     Strans  16.408942    12
6     ALA_N    SOL_O        1.0       Srot   0.637448    12
7     ALA_N    SOL_O        1.0      count   0.600000    12
8     ALA_O    SOL_O        1.0  Sor_test2   0.866724   108
9     ALA_O    SOL_O        1.0     Strans  11.186391   108
10    ALA_O    SOL_O        1.0       Srot   0.579712   108
11    ALA_O    SOL_O        1.0      count   5.400000   108
12    ARG_C    SOL_O        1.0  Sor_test2   3.136063   502
13    ARG_C    SOL_O        1.0     Strans  12.455572   502
14    ARG_C    SOL_O        1.0       Srot   0.505180   502
15    ARG_C    SOL_O        1.0      cou

In [82]:
ua_solvent_df.to_csv('ua_solvent.csv')

In [51]:
S_or_ua=0
S_trans_ua=0
S_rot_ua=0
for i in range (0,235,4):
    S_or_ua+= ua_solvent_df['value'][i] #4k -> orientational entropy
    S_trans_ua+= ua_solvent_df['value'][i+1] #4k+1 -> translational entropy
    S_rot_ua+= ua_solvent_df['value'][i+2] #4k+2 -> rotational entropy

In [52]:
print(S_or_ua)
print(S_trans_ua)
print(S_rot_ua)

68.4043269733432
789.3224281624549
51.92750979994863


In [53]:
S_total_ua=S_or_ua+S_trans_ua+S_rot_ua
print(S_total_ua)

909.6542649357468


In [54]:
ua_solute_df=result_ua['atomLevel']['soluteData']

In [55]:
print(ua_solute_df.to_string())

    resName      variable       value count
0     ALA_N     WM_Strans    2.237692   240
1     ALA_N       WM_Srot    1.842312   240
2     ALA_N  WM_UA_Strans    0.003090   240
3     ALA_N    WM_UA_Srot    2.528334   240
4     ARG_N     WM_Strans    6.307841   220
5     ARG_N       WM_Srot    4.186435   220
6     ARG_N  WM_UA_Strans    4.421717   220
7     ARG_N    WM_UA_Srot    1.332549   220
8     ARG_N       conf_AE   42.385285   220
9     ASN_N     WM_Strans    4.764232   280
10    ASN_N       WM_Srot    2.999115   280
11    ASN_N  WM_UA_Strans    2.109526   280
12    ASN_N    WM_UA_Srot    0.467818   280
13    ASN_N       conf_AE   24.279920   280
14    ASP_N     WM_Strans    3.990985   140
15    ASP_N       WM_Srot    2.564136   140
16    ASP_N  WM_UA_Strans    1.725408   140
17    ASP_N    WM_UA_Srot    0.418705   140
18    ASP_N       conf_AE   25.935006   140
19    CL_CL     WM_Strans   21.495253   160
20    CL_CL       WM_Srot    0.000000   160
21    CL_CL  WM_UA_Strans    0.0

In [77]:
ua_solute_df.to_csv('ua_solute.csv')

## Solute Contact

In [57]:
result_solcon = poseidon_object.run_analysis(level_list = ['soluteContacts'], verbose = False, forceUnits="Kcal") # forces value supplied in the trajectory are in kcal
print(result_solcon)

2023-09-13 10:52:11.609789

solvent: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

water: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

1. Populate Dictionaries

memory use: 1.398 GB
0:00:00.001668
memory use: 1.398 GB
0:00:02.960307
0:00:02.960782
memory use: 1.398 GB

Total number of frames: 20.0
Number of atoms in each frame: 11645
Number of variables in each list: 19

2. Process Dictionaries
['soluteContacts']
---level: soluteContacts


/home/ioana/Desktop/STFC/CodeEntropy/code/CodeEntropy/CodeEntropy/poseidon/analysis/EECalculation.py:1515: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  solventData = pd.concat([solventData, newRowSolvent], ignore_index=True)




0:00:11.689111
{'soluteContacts': {'solventData':               nearest assigned  shell_num   variable      value count
0     ALA_107_ALA_107      SOL        1.0  Sor_test2   0.483728    13
1     ALA_107_ALA_107      SOL        1.0     Strans  12.249477    13
2     ALA_107_ALA_107      SOL        1.0       Srot   1.600586    13
3     ALA_107_ALA_107      SOL        1.0      count   0.650000    13
4     ALA_107_ARG_112      SOL        1.0  Sor_test2   0.173337    18
...               ...      ...        ...        ...        ...   ...
5691    VAL_99_TYR_23      SOL        1.0      count   0.200000     4
5692    VAL_99_VAL_99      SOL        1.0  Sor_test2   0.000000     7
5693    VAL_99_VAL_99      SOL        1.0     Strans  21.266607     7
5694    VAL_99_VAL_99      SOL        1.0       Srot   1.604742     7
5695    VAL_99_VAL_99      SOL        1.0      count   0.350000     7

[5696 rows x 6 columns], 'soluteData': Empty DataFrame
Columns: [resName, variable, value, count]
Index: []

In [58]:
solcon_df=result_solcon['soluteContacts']['solventData']

In [62]:
print(solcon_df.to_string())

               nearest assigned  shell_num   variable      value count
0      ALA_107_ALA_107      SOL        1.0  Sor_test2   0.483728    13
1      ALA_107_ALA_107      SOL        1.0     Strans  12.249477    13
2      ALA_107_ALA_107      SOL        1.0       Srot   1.600586    13
3      ALA_107_ALA_107      SOL        1.0      count   0.650000    13
4      ALA_107_ARG_112      SOL        1.0  Sor_test2   0.173337    18
5      ALA_107_ARG_112      SOL        1.0     Strans  10.911429    18
6      ALA_107_ARG_112      SOL        1.0       Srot   1.265743    18
7      ALA_107_ARG_112      SOL        1.0      count   0.900000    18
8      ALA_107_ASN_103      SOL        1.0  Sor_test2   0.000000    20
9      ALA_107_ASN_103      SOL        1.0     Strans  16.497932    20
10     ALA_107_ASN_103      SOL        1.0       Srot   0.941189    20
11     ALA_107_ASN_103      SOL        1.0      count   1.000000    20
12     ALA_107_ASP_101      SOL        1.0  Sor_test2   0.000000    11
13    

In [78]:
solcon_df.to_csv('solcon.csv')

In [63]:
S_or_solcon=0
S_trans_solcon=0
S_rot_solcon=0
for i in range (0,5695,4):
    S_or_solcon+= solcon_df['value'][i] #4k -> orientational entropy
    S_trans_solcon+= solcon_df['value'][i+1] #4k+1 -> translational entropy
    S_rot_solcon+= solcon_df['value'][i+2] #4k+2 -> rotational entropy

In [64]:
print(S_or_solcon)
print(S_trans_solcon)
print(S_rot_solcon)

125.96856875107954
23530.467734182486
3999.1662262191276


In [65]:
S_total_solcon=S_or_solcon+S_trans_solcon+S_rot_solcon
print(S_total_solcon)

27655.602529152693


In [66]:
S_total=S_total_wm+S_total_res+S_total_ua+S_total_solcon
print(S_total) #kcal mol-1 K-1

31019.959727716065


In [67]:
S_total_kJ=S_total*4.184

In [68]:
print(S_total_kJ)

129787.51150076401
